In [1]:
import os
import h5py
import sv as SV
import numpy as np
import astropy.table as atable
from astropy.cosmology import FlatLambdaCDM

from desitarget.sv1.sv1_targetmask import bgs_mask as sv1_bgs_mask
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask
from desitarget.targetmask import bgs_mask as main_bgs_mask

In [2]:
# -- plotting -- 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

In [3]:
dir_fuji = '/global/cfs/cdirs/desi/spectro/redux/fuji/zcatalog/'
dir_guad = '/global/cfs/cdirs/desi/spectro/redux/guadalupe/zcatalog/'
sv1 = atable.Table.read(os.path.join(dir_fuji, 'ztile-sv1-bright-cumulative.fits'))
sv3 = atable.Table.read(os.path.join(dir_fuji, 'ztile-sv3-bright-cumulative.fits'))
main = atable.Table.read(os.path.join(dir_guad, 'ztile-main-bright-cumulative.fits'))

In [4]:
sv3[:2]

<Table length=2>
     TARGETID     LASTNIGHT SPGRPVAL ... TSNR2_LRG ZCAT_NSPEC ZCAT_PRIMARY
      int64         int32    int32   ...  float32    int16        bool    
----------------- --------- -------- ... --------- ---------- ------------
39632946289707054  20210405 20210405 ... 11.665173          5        False
39632951343844427  20210405 20210405 ... 12.735361          2        False

In [5]:
sv1_bright = (sv1['SV1_BGS_TARGET'] & sv1_bgs_mask['BGS_BRIGHT']) != 0 
sv1_faint  = (sv1['SV1_BGS_TARGET'] & sv1_bgs_mask['BGS_FAINT']) != 0 

sv3_bright = (sv3['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_BRIGHT']) != 0 
sv3_faint  = (sv3['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_FAINT']) != 0 

main_bright  = (main['BGS_TARGET'] & main_bgs_mask['BGS_FAINT']) != 0 
main_faint  = (main['BGS_TARGET'] & main_bgs_mask['BGS_FAINT']) != 0 

In [6]:
sv1_cuts = ((sv1['COADD_FIBERSTATUS'] == 0) & (sv1['Z'] != 1e20) & (sv1['ZWARN'] == 0) & (sv1['DELTACHI2'] > 40))
print(np.mean(sv1_cuts))
sv3_cuts = ((sv3['COADD_FIBERSTATUS'] == 0) & (sv3['Z'] != 1e20) & (sv3['ZWARN'] == 0) & (sv3['DELTACHI2'] > 40))
print(np.mean(sv3_cuts))
main_cuts = ((main['COADD_FIBERSTATUS'] == 0) & (main['Z'] != 1e20) & (main['ZWARN'] == 0) & (main['DELTACHI2'] > 40))
print(np.mean(main_cuts))

0.6850104384133612
0.7030595964335993
0.791060212514758


In [7]:
print('SV1: BRIGHT %i FAINT %i ALL %i redshifts' % (np.sum(sv1_bright & sv1_cuts), np.sum(sv1_faint & sv1_cuts), np.sum(sv1_bright & sv1_cuts) + np.sum(sv1_faint & sv1_cuts)))
print('SV3: BRIGHT %i FAINT %i ALL %i redshifts' % (np.sum(sv3_bright & sv3_cuts), np.sum(sv3_faint & sv3_cuts), np.sum(sv3_bright & sv3_cuts) + np.sum(sv3_faint & sv3_cuts)))
print('MAIN: BRIGHT %i FAINT %i ALL %i redshifts' % (np.sum(main_bright & main_cuts), np.sum(main_faint & main_cuts), np.sum(main_bright & main_cuts) + np.sum(main_faint & main_cuts)))

print('%i total BRIGHT SUCCESSFUL REDSHIFTS' % (np.sum(sv1_bright & sv1_cuts) + np.sum(sv3_bright & sv3_cuts)))
print('%i total FAINT SUCCESSFUL REDSHIFTS' % (np.sum(sv1_faint & sv1_cuts) + np.sum(sv3_faint & sv3_cuts)))

SV1: BRIGHT 50600 FAINT 36767 ALL 87367 redshifts
SV3: BRIGHT 234735 FAINT 164765 ALL 399500 redshifts
MAIN: BRIGHT 266398 FAINT 266398 ALL 532796 redshifts
285335 total BRIGHT SUCCESSFUL REDSHIFTS
201532 total FAINT SUCCESSFUL REDSHIFTS


In [8]:
def stephist(nh, _edges):
    bin_edges = np.sort(list(_edges) + list(_edges[1:-1]))
    return bin_edges, np.array([[x, x] for x in nh]).flatten()

In [9]:
#gama_dr3 = atable.Table.read('/global/cscratch1/sd/chahah/bgs-cmxsv/sv-paper/gama_specobj.fits')
gama_dr4 = atable.Table.read('/global/cscratch1/sd/chahah/bgs-cmxsv/gama_dr4/TilingCatv46.fits')

gama_dr4 = gama_dr4[(gama_dr4['NQ'] >= 3) & (gama_dr4['SURVEY_CLASS'] >= 3)]

In [10]:
fig = plt.figure(figsize=(12,6))
sub = fig.add_subplot(121)

nh, edges = np.histogram(sv1['Z'][sv1_bright & sv1_cuts], range=(0., 0.6), bins=30)
bin_edges, nhist0 = stephist(nh, edges)

sub.fill_between(bin_edges, np.zeros(len(bin_edges)), nhist0, facecolor='C0', hatch='/')
sub.plot(bin_edges, nhist0, c='k')


nh, edges = np.histogram(sv3['Z'][sv3_bright & sv3_cuts], range=(0., 0.6), bins=30)
bin_edges, nhist1 = stephist(nh, edges)
sub.fill_between(bin_edges, nhist0, nhist0 + nhist1, facecolor='C0')

nh, edges = np.histogram(gama_dr4['Z'], range=(0., 0.7), bins=35)
bin_edges, nhist0 = stephist(nh, edges)
sub.plot(bin_edges, nhist0, c='k', lw=0.75, linestyle='-.', label='GAMA DR4')


sub.text(0.05, 0.95, 'BGS Bright', ha='left', va='top', transform=sub.transAxes, fontsize=25)
sub.legend(loc='center right', fontsize=20, handletextpad=0.1)
sub.set_xlabel('$z$', fontsize=25)
sub.set_xlim(0., 0.7)
sub.set_ylabel(r'${\rm d}N/{\rm d}z$ $\Delta z$', fontsize=25)
sub.set_ylim(0., 3.e4)

sub = fig.add_subplot(122)

nh, edges = np.histogram(sv1['Z'][sv1_faint & sv1_cuts], range=(0., 0.7), bins=35)
bin_edges, nhist0 = stephist(nh, edges)

sub.fill_between(bin_edges, np.zeros(len(bin_edges)), nhist0, facecolor='C1', hatch='/', label='SV1')
sub.plot(bin_edges, nhist0, c='k')

nh, edges = np.histogram(sv3['Z'][sv3_faint & sv3_cuts], range=(0., 0.7), bins=35)
bin_edges, nhist1 = stephist(nh, edges)
sub.fill_between(bin_edges, nhist0, nhist0 + nhist1, facecolor='C1', label=r'$1\%$')


sub.text(0.05, 0.95, 'BGS Faint', ha='left', va='top', transform=sub.transAxes, fontsize=25)
#sub.text(0.95, 0.95, '1\%', ha='right', va='top', transform=sub.transAxes, fontsize=25)

sub.legend(loc='upper right', handletextpad=0.2, fontsize=25)
sub.set_xlabel('redshift ($z$)', fontsize=25)
sub.set_xlim(0., 0.7)
sub.set_yticklabels([])
sub.set_ylim(0., 3.0e4)

fig.subplots_adjust(wspace=0.1)
fig.savefig('figs/dndz.pdf', bbox_inches='tight')

In [22]:
nh, edges = np.histogram(sv1['Z'][sv1_bright & sv1_cuts], range=(0., 0.6), bins=30)

In [27]:
nh

array([1117,  996, 1548, 2073, 2595, 2774, 3536, 3146, 4061, 3435, 3271,
       3183, 3282, 2752, 2248, 2046, 1772, 1357, 1187,  855,  693,  620,
        403,  242,  190,  126,   72,   36,   24,   25])

In [28]:
nh_sv1, edges = np.histogram(sv1['Z'][sv1_bright & sv1_cuts], range=(0., 0.6), bins=30)
nh_sv3, edges = np.histogram(sv3['Z'][sv3_bright & sv3_cuts], range=(0., 0.6), bins=30)
np.savetxt('dat/fig18_dNdz_dz_bgsbright.dat', 
           np.vstack([edges[:-1], edges[1:], nh_sv1, nh_sv3]).T, 
           fmt='%f %f %i %i', header='dN/dz * dz\n z_low, z_high, N_SV1, N_SV3')

nh_sv1, edges = np.histogram(sv1['Z'][sv1_faint & sv1_cuts], range=(0., 0.7), bins=35)
nh_sv3, edges = np.histogram(sv3['Z'][sv3_faint & sv3_cuts], range=(0., 0.7), bins=35)
np.savetxt('dat/fig18_dNdz_dz_bgsfaint.dat', 
           np.vstack([edges[:-1], edges[1:], nh_sv1, nh_sv3]).T, 
           fmt='%f %f %i %i', header='dN/dz * dz\n z_low, z_high, N_SV1, N_SV3')

In [11]:
zlow, zhigh, volz = np.loadtxt('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/everest/LSScats/2.1/BGS_ANY_nz.dat', skiprows=2, unpack=True, usecols=[1, 2, 5])

In [12]:
zlim = (zlow < 0.7)

zlow = zlow[zlim]
zhigh = zhigh[zlim]
volz = volz[zlim]

In [13]:
nz_all, zedges = np.histogram(sv3['Z'][sv3_cuts], bins=np.array(list(zlow)+[zhigh[-1]]))
nz_bright, _ = np.histogram(sv3['Z'][sv3_bright & sv3_cuts], bins=np.array(list(zlow)+[zhigh[-1]]))
nz_faint, _ = np.histogram(sv3['Z'][sv3_faint & sv3_cuts], bins=np.array(list(zlow)+[zhigh[-1]]))

nz_all = nz_all.astype(float) / volz
nz_bright = nz_bright.astype(float) / volz
nz_faint = nz_faint.astype(float) / volz

In [14]:
nz_all

array([3.90392664e-01, 1.27622677e-01, 8.77224941e-02, 9.55205385e-02,
       4.89250907e-02, 5.23638401e-02, 5.34683607e-02, 3.08226437e-02,
       3.16298228e-02, 2.52611939e-02, 2.32743712e-02, 2.07396462e-02,
       2.23050638e-02, 1.51172246e-02, 1.19508633e-02, 9.66653689e-03,
       7.36603144e-03, 5.81243616e-03, 4.81524570e-03, 3.64153936e-03,
       3.19380156e-03, 2.23671197e-03, 1.52200759e-03, 1.16241342e-03,
       8.30123485e-04, 6.33734194e-04, 4.12875588e-04, 3.05213519e-04,
       1.63014499e-04, 1.19362051e-04, 7.56659667e-05, 5.30910575e-05,
       2.65796114e-05, 2.08349893e-05, 1.05418749e-05])

In [21]:
np.savetxt('dat/fig19_nz_bgs_svda.dat', np.vstack([zedges[:-1], zedges[1:], nz_all, nz_bright, nz_faint]).T, header='zlow, zhigh, nz_all, nz_bright, nz_faint')

In [16]:
# DESI LRG n(z)
#nz_lrg = np.loadtxt('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/everest/LSScats/2.1/LRG_main_nz.dat', skiprows=2, unpack=True, usecols=[3])
zmid_lrg, zlow_lrg, zhigh_lrg, nz_lrg, vol_lrg = np.loadtxt('/global/cfs/cdirs/desi/survey/catalogs/DA02/LSS/everest/LSScats/1/LRGzdone_nz.dat', skiprows=2, unpack=True, usecols=[0, 1, 2, 3, 5])
vol_lrg /= 1130.138 

zlo_lrg, zhi_lrg, N_lrg = np.loadtxt('main_lrg_dndz.txt', unpack=True, skiprows=1, usecols=[0, 1, 2])
_zmid = 0.5 * (zlo_lrg[1::2] + zhi_lrg[2::2])
N_lrg = N_lrg[1::2] + N_lrg[2::2]

nz_lrg = N_lrg[_zmid < 0.7] / vol_lrg[zmid_lrg < 0.7]
nz_lrg[zmid_lrg[zmid_lrg < 0.7] < 0.4] = 0.

zedges_lrg = np.concatenate([[zlow_lrg[0]], zhigh_lrg[zmid_lrg < 0.7]])

In [17]:
# SDSS MGS n(z)
sdss_dr16 = atable.Table.read('/global/cscratch1/sd/chahah/bgs-cmxsv/sv-paper/specObj-dr16.fits')
is_mgs = ((sdss_dr16['LEGACY_TARGET1']  & (64 | 128 | 256)) > 0)
nz_mgs, _ = np.histogram(sdss_dr16['Z'][is_mgs], bins=np.array(list(zlow)+[zhigh[-1]]))
nz_mgs = nz_mgs.astype(float) / (volz / 168.8 * 7500.)

In [18]:
# GAMA n(z) (assuming DR4 footprint is the same as DR3)
area_gama = (#(8.6 * 6.53) + #G02
    (12. * 5.0) + #G09
    (12. * 4.0) + #G12
    (12. * 4.0)) #G15

nz_gama, _ = np.histogram(gama_dr4['Z'], bins=np.array(list(zlow)+[zhigh[-1]]))
nz_gama = nz_gama.astype(float) / (volz / 168.8 * area_gama)

In [19]:
plt.scatter(gama_dr4['RA'], gama_dr4['DEC'], s=1)

In [20]:
fig = plt.figure(figsize=(10,8))
sub = fig.add_subplot(111)
bin_edges, nhist = stephist(nz_mgs, zedges)
_sdss, = sub.plot(bin_edges, nhist, c='k', ls='--', lw=1)#, label='SDSS MGS')
bin_edges, nhist = stephist(nz_gama, zedges)
_gama, = sub.plot(bin_edges, nhist, c='k', ls='-.', lw=1)#, label='SDSS MGS')
_lowz = sub.axhline(3e-4, color='k', linestyle=':')#, label='BOSS LOWZ')


sub.plot([], [], c='k', lw=2, label='BGS ($1\%$)')

bin_edges, nhist = stephist(nz_bright, zedges)
sub.plot(bin_edges, nhist, c='C0', lw=2, label='BGS Bright')

bin_edges, nhist = stephist(nz_faint, zedges)
sub.plot(bin_edges, nhist, c='C1', lw=2, label='BGS Faint')

bin_edges, nhist = stephist(nz_all, zedges)
sub.plot(bin_edges, nhist, c='k', lw=2)

bin_edges, nhist = stephist(nz_lrg, zedges_lrg)
sub.plot(bin_edges, nhist, c='r', lw=0.75, label='DESI LRG')


legend1 = sub.legend(loc='upper right', handletextpad=0.2, fontsize=25)
fig.gca().add_artist(legend1)
legend2 = sub.legend([_sdss, _lowz, _gama], ['SDSS MGS', 'BOSS LOWZ', 'GAMA DR4'], loc='lower left', handletextpad=0.2, frameon=True, fontsize=20)
fig.gca().add_artist(legend2)


sub.set_xlabel('$z$', fontsize=25)
sub.set_xlim(0., 0.7)
sub.set_ylabel(r'$\bar{n}(z)$ [$({\rm Mpc}/h)^{-3}$]', fontsize=25)
sub.set_yscale('log')
sub.set_ylim(2e-5, None)
fig.savefig('figs/nofz.pdf', bbox_inches='tight')

In [ ]:


sub.plot([], [], c='k', lw=2, label='BGS ($1\%$)')

bin_edges, nhist = stephist(nz_bright, zedges)
sub.plot(bin_edges, nhist, c='C0', lw=2, label='BGS Bright')

bin_edges, nhist = stephist(nz_faint, zedges)
sub.plot(bin_edges, nhist, c='C1', lw=2, label='BGS Faint')

bin_edges, nhist = stephist(nz_all, zedges)
sub.plot(bin_edges, nhist, c='k', lw=2)

bin_edges, nhist = stephist(nz_lrg, zedges_lrg)
sub.plot(bin_edges, nhist, c='r', lw=0.75, label='DESI LRG')


legend1 = sub.legend(loc='upper right', handletextpad=0.2, fontsize=25)
fig.gca().add_artist(legend1)
legend2 = sub.legend([_sdss, _lowz, _gama], ['SDSS MGS', 'BOSS LOWZ', 'GAMA DR4'], loc='lower left', handletextpad=0.2, frameon=True, fontsize=20)
fig.gca().add_artist(legend2)


sub.set_xlabel('$z$', fontsize=25)
sub.set_xlim(0., 0.7)
sub.set_ylabel(r'$\bar{n}(z)$ [$({\rm Mpc}/h)^{-3}$]', fontsize=25)
sub.set_yscale('log')
sub.set_ylim(2e-5, None)
fig.savefig('figs/nofz.pdf', bbox_inches='tight')

In [22]:
fig = plt.figure(figsize=(10,8))
sub = fig.add_subplot(111)
# bin_edges, nhist = stephist(nz_mgs, zedges)
# _sdss, = sub.plot(bin_edges, nhist, c='k', ls='--', lw=1)#, label='SDSS MGS')
# bin_edges, nhist = stephist(nz_gama, zedges)
# _gama, = sub.plot(bin_edges, nhist, c='k', ls='-.', lw=1)#, label='SDSS MGS')
# # _lowz = sub.axhline(3e-4, color='k', linestyle=':')#, label='BOSS LOWZ')


sub.plot([], [], c='k', lw=2, label='BGS ($1\%$)')

bin_edges, nhist = stephist(nz_all, zedges)
sub.plot(bin_edges, nhist, c='k', lw=2)

bin_edges, nhist = stephist(nz_bright, zedges)
sub.plot(bin_edges, nhist, c='C0', lw=2, label='BGS Bright')

bin_edges, nhist = stephist(nz_faint, zedges)
sub.plot(bin_edges, nhist, c='C1', lw=2, label='BGS Faint')


legend1 = sub.legend(loc='upper right', handletextpad=0.2, fontsize=25)
fig.gca().add_artist(legend1)
legend2 = sub.legend([_sdss, _lowz, _gama], ['SDSS MGS', 'BOSS LOWZ', 'GAMA DR3'], loc='lower left', handletextpad=0.2, frameon=True, fontsize=20)
fig.gca().add_artist(legend2)

sub.set_xlabel('$z$', fontsize=25)
sub.set_xlim(0., 0.7)
sub.set_ylabel(r'$\bar{n}(z)$ [$({\rm Mpc}/h)^{-3}$]', fontsize=25)
sub.set_yscale('log')
sub.set_ylim(2e-5, 1.)

(2e-05, 1.0)